In [0]:
-- Create log table for performance testing in Databricks
-- Catalog: gap_catalog, Schema: log
DROP table IF EXISTS gap_catalog.log.performance_test_log;
CREATE TABLE IF NOT EXISTS gap_catalog.log.performance_test_log (
    test_case_name STRING NOT NULL COMMENT 'Name of the test case being executed',
    start_time TIMESTAMP NOT NULL COMMENT 'Start time of the test execution',
    end_time TIMESTAMP COMMENT 'End time of the test execution',
    compute_size STRING COMMENT 'Size of the compute cluster used for testing',
    started_by STRING NOT NULL COMMENT 'User who initiated the test',
    status STRING NOT NULL COMMENT 'Status of the test (RUNNING, COMPLETED, FAILED, etc.)',
    process_key STRING COMMENT 'Unique identifier for the test process',
    record_count BIGINT COMMENT 'Number of records processed during the test',
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP() COMMENT 'Record creation timestamp',
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP() COMMENT 'Record last update timestamp'
)
USING DELTA
TBLPROPERTIES (
    'delta.feature.allowColumnDefaults' = 'supported',
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true'
)
COMMENT 'Performance testing log table to track test executions and metrics';

-- Create logging procedures for performance testing

-- Procedure to start a performance test log entry
CREATE OR REPLACE PROCEDURE gap_catalog.log.start_performance_test(
    IN test_case_name STRING,
    IN compute_size STRING,
    IN started_by STRING,
    IN process_key STRING
)
LANGUAGE SQL
SQL SECURITY INVOKER
COMMENT 'Procedure to log the start of a performance test'
AS
BEGIN
    INSERT INTO gap_catalog.log.performance_test_log (
        test_case_name,
        start_time,
        compute_size,
        started_by,
        status,
        process_key,
        created_at,
        updated_at
    )
    VALUES (
        test_case_name,
        CURRENT_TIMESTAMP(),
        compute_size,
        started_by,
        'RUNNING',
        process_key,
        CURRENT_TIMESTAMP(),
        CURRENT_TIMESTAMP()
    );
    
    SELECT CONCAT('Performance test started for: ', test_case_name, ' with process key: ', process_key) as message;
END;

-- Procedure to complete a performance test log entry
CREATE OR REPLACE PROCEDURE gap_catalog.log.complete_performance_test(
    IN process_key STRING,
    IN status STRING,
    IN record_count BIGINT DEFAULT NULL
)
LANGUAGE SQL
SQL SECURITY INVOKER
COMMENT 'Procedure to log the completion of a performance test'
AS
BEGIN
    UPDATE gap_catalog.log.performance_test_log
    SET 
        end_time = CURRENT_TIMESTAMP(),
        status = status,
        record_count = record_count,
        updated_at = CURRENT_TIMESTAMP()
    WHERE process_key = process_key
      AND end_time IS NULL;
    
    /*IF ROW_COUNT() = 0 THEN
        SELECT CONCAT('No running test found with process key: ', process_key) as message;
    ELSE
        SELECT CONCAT('Performance test completed with status: ', status, ' for process key: ', process_key) as message;
    END IF;*/
END;

-- Procedure to log a complete performance test in one call
CREATE OR REPLACE PROCEDURE gap_catalog.log.log_performance_test(
    IN test_case_name STRING,
    IN start_time TIMESTAMP,
    IN end_time TIMESTAMP,
    IN compute_size STRING,
    IN started_by STRING,
    IN status STRING,
    IN process_key STRING,
    IN record_count BIGINT DEFAULT NULL
)
LANGUAGE SQL
SQL SECURITY INVOKER
COMMENT 'Procedure to log a complete performance test execution'
AS
BEGIN
    INSERT INTO gap_catalog.log.performance_test_log (
        test_case_name,
        start_time,
        end_time,
        compute_size,
        started_by,
        status,
        process_key,
        record_count,
        created_at,
        updated_at
    )
    VALUES (
        test_case_name,
        start_time,
        end_time,
        compute_size,
        started_by,
        status,
        process_key,
        record_count,
        CURRENT_TIMESTAMP(),
        CURRENT_TIMESTAMP()
    );
    
    SELECT CONCAT('Performance test logged successfully for: ', test_case_name) as message;
END;

-- Procedure to update test status
CREATE OR REPLACE PROCEDURE gap_catalog.log.update_test_status(
    IN process_key STRING,
    IN new_status STRING
)
LANGUAGE SQL
SQL SECURITY INVOKER
COMMENT 'Procedure to update the status of a running performance test'
AS
BEGIN
    UPDATE gap_catalog.log.performance_test_log
    SET 
        status = new_status,
        updated_at = CURRENT_TIMESTAMP()
    WHERE process_key = process_key;
    
    IF ROW_COUNT() = 0 THEN
        SELECT CONCAT('No test found with process key: ', process_key) as message;
    ELSE
        SELECT CONCAT('Test status updated to: ', new_status, ' for process key: ', process_key) as message;
    END IF;
END;

---------------------------------------------------------------------------------
-- Usage examples for the performance testing log procedures

-- Example 1: Start a performance test
CALL gap_catalog.log.start_performance_test(
    test_case_name => 'ADS_RDS_Q',
    compute_size => '2XS',
    started_by => 'FilipOliva',
    process_key => '889'
);

-- Example 2: Complete a performance test
CALL gap_catalog.log.complete_performance_test(
    process_key => '889',  -- process_key from start procedure
    status => 'COMPLETED',
    record_count => 1000000  -- record_count
);

-- Query to view performance test logs
SELECT 
    test_case_name,
    start_time,
    end_time,
    CASE 
        WHEN end_time IS NOT NULL THEN 
            CAST((UNIX_TIMESTAMP(end_time) - UNIX_TIMESTAMP(start_time)) AS STRING) || ' seconds'
        ELSE 'Still running'
    END as duration,
    compute_size,
    started_by,
    status,
    process_key,
    record_count,
    created_at
FROM gap_catalog.log.performance_test_log
ORDER BY created_at DESC;

------
CALL gap_catalog.ads_etl_owner.sp_ADS_RDS_EVENT_TYPES_MEP(
    '2025-08-16',  -- p_load_date
    '12345'        -- p_process_key
);
SELECT ROW_COUNT() as rows_affected;